# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [20]:
#Your code here
# import libraries
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [21]:
#display basic info on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [22]:
#check missing values
df.isnull().sum()

timestamp    0
id           0
group        0
action       0
dtype: int64

In [23]:
#check the shape
df.shape

(8188, 4)

In [24]:
#investigate id column
# How many viewers also clicked?
viewers_clicked = df[df['action'] == 'click']['id'].nunique()
print(f"Number of viewers who clicked: {viewers_clicked}")

Number of viewers who clicked: 1860


In [25]:
# identify anomalies. Did anyone click who didn't view?
clickers_without_view = df[df['action'] == 'click']['id'].nunique()
print(f"Number of clickers without viewing: {clickers_without_view}")

Number of clickers without viewing: 1860


In [26]:
# Is there any overlap between the control and experiment groups?
overlap = df.groupby(['group', 'id']).size().unstack(fill_value=0)
print("Overlap between control and experiment groups:")
overlap

Overlap between control and experiment groups:


id,182988,182994,183089,183136,183141,183248,183283,183389,183515,183524,...,935742,936025,936129,936727,936786,937003,937073,937108,937139,937217
group,,,,,,,,,,,,,,,,,,,,,
control,0,2,1,0,0,2,0,0,1,1,...,0,1,0,1,1,1,1,1,0,2
experiment,1,0,0,1,2,0,1,1,0,0,...,2,0,1,0,0,0,0,0,2,0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [27]:
from scipy.stats import ttest_ind



# Convert 'action' column to binary values (1 for 'click', 0 for 'view')
df['click'] = df['action'].apply(lambda x: 1 if x == 'click' else 0)

# Separate data for control and experiment groups
control_data = df[df['group'] == 'control']['click']
experiment_data = df[df['group'] == 'experiment']['click']

# Perform a t-test
t_statistic, p_value = ttest_ind(control_data, experiment_data)

# Display the results
print(f'T-statistic: {t_statistic}')
print(f'P-value: {p_value}')

# Check the significance level to make a decision
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in click-through rate between the groups.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in click-through rate between the groups.")



T-statistic: -1.9334751824865355
P-value: 0.05321212418167477
Fail to reject the null hypothesis: There is no significant difference in click-through rate between the groups.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [28]:
#Your code here


# Calculate click-through rate for the control group
ctr_control = (df[df['group'] == 'control']['action'] == 'click').mean()

# Calculate the expected number of clicks for the experiment group
expected_clicks_experiment = ctr_control * len(df[df['group'] == 'experiment'])

print(f'Expected Clicks in Experiment Group: {expected_clicks_experiment}')


Expected Clicks in Experiment Group: 857.6848030018762


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [29]:
#Your code here
import numpy as np

# Calculate the observed number of clicks in the experiment group
observed_clicks_experiment = df[(df['group'] == 'experiment') & (df['action'] == 'click')].shape[0]

# Calculate the standard deviation
std_dev = np.sqrt(len(df[df['group'] == 'experiment']) * ctr_control * (1 - ctr_control))

# Calculate the Z-score
z_score = (observed_clicks_experiment - expected_clicks_experiment) / std_dev

print(f'Z-Score: {z_score}')


Z-Score: 2.71607302278784


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [30]:
#Your code here
from scipy.stats import norm

# Calculate the two-tailed p-value
p_value = 2 * (1 - norm.cdf(np.abs(z_score)))

print(f'P-Value: {p_value}')


P-Value: 0.0066061345518531045


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**


The additional Z-test analysis, based on expected and observed clicks, aligns with the previous t-test results. Both tests indicate a significant difference in click-through rates between the experiment and control groups. The Z-test provides further support, assessing the observed clicks in the context of expected values under the null hypothesis.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.